# Gradient Descent

1. [10 points] Write a version of gradient descent that chooses n random starting points and outputs the parameters resulting in minimum training loss across all the runs.

In [8]:
import numpy as np
import torch
class LeastSquaresGradient:
        
    # fit the model to the data
    def fit(self, X, y, w0, alpha, h, tolerance, max_iterations):

        # save the training data
        X = np.hstack((np.ones([X.shape[0], 1]), X))
        
        # find the w values that minimize the sum of squared errors via gradient descent
        L = lambda w: ((X @ w).T - y.T) @ (X @ w - y)
        self.w = self.gradientDescent(L, w0, alpha, h, tolerance, max_iterations)
                
    # predict the output from testing data
    def predict(self, X):        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # return the predicted y's
        return X @ self.w

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, alpha, h, tolerance, max_iterations):
        # set x equal to the initial guess
        x = x0

        # take up to maxIterations number of steps
        for counter in range(max_iterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == max_iterations - 1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            x -= alpha*gradient
            
    # estimate the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)
        
        # compute f at current point
        fx = f(x)

        # find each component of the gradient
        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - fx)/h

        # return the gradient
        return gradient

    




ModuleNotFoundError: No module named 'pytorch'

In [13]:
# check model

X = np.array([[6], [7], [8], [9], [7]])
y = np.array([1, 2, 3, 3, 4])

# instantiate an least squares object, fit to data, predict data
model = LeastSquaresGradient()

print('Fitting the model...\n')
model.fit(X, y, w0 = np.random.rand(X.shape[1]+1), alpha=0.001, h=0.001, tolerance=0.01, max_iterations=100000)

predictions = model.predict(X)

# print the predictions
print('\nThe predicted y values are', np.round(predictions, 2))

# print the real y values
print('The real y values are', y)

# print the w values
parameters = model.w
print('The w values are', parameters)

Fitting the model...

Gradient descent took 18109 iterations to converge
The norm of the gradient is 0.009999497957967543

The predicted y values are [1.89 2.4  2.91 3.41 2.4 ]
The real y values are [1 2 3 3 4]
The w values are [-1.1588022   0.50801263]


2. [5 points] Write a version of gradient descent that uses momentum.

In [20]:

# estimate the gradient
def computeGradientWithMomentum(f, x, h, pastGradient):
    n = len(x)
    gradient = np.zeros(n)
    pastGradient = np.array(pastGradient)
    for counter in range(n):
        xUp = x.copy()
        xUp[counter] += h
        gradient[counter] = (f(xUp) - f(x))/h - pastGradient[counter]*0.02
            
    return gradient, pastGradient

    # run gradient descent and output the coordinates of the estimated critical point
def gradientDescent(f, x0, alpha, h, tolerance, maxIterations):
    # set x equal to the initial guess
    x = x0
    # make a pastGradient parameter to store previous gradients
    pastGradient = np.zeros(x)
    # take up to maxIterations number of steps
    for counter in range(maxIterations):
        # update the gradient and store the previous gradient
        gradient,pastGradient = computeGradientWithMomentum(f, x, h, pastGradient)
        pastGradient = gradient
        # stop if the norm of the gradient is near 0 (success)
        if np.linalg.norm(gradient) < tolerance:
            print('Gradient descent took', counter, 'iterations to converge')
            print('The norm of the gradient is', np.linalg.norm(gradient))
            
            # return the approximate critical point x
            return x
            
        # print a message if we do not converge (failure)
        elif counter == maxIterations-1:
            print("Gradient descent failed")
            print('The gradient is', gradient)
                
            # return x, sometimes it is still pretty good
            return x
            
        # take a step in the opposite direction as the gradient
        x -= alpha*gradient

In [23]:
# test sin(x)
f = lambda x: np.sin(x)

x = gradientDescent(f,[2],0.5,0.5,0.0001,10000)
print(x, f(x))

Gradient descent took 18 iterations to converge
The norm of the gradient is 9.976123420763023e-05
[4.46228432] [-0.96888652]


/var/folders/b8/xxt8cq0j5s50gztcml150m700000gn/T/ipykernel_10571/2941004565.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gradient[counter] = (f(xUp) - f(x))/h - pastGradient[counter]*0.02


In [22]:
# test x^2
f = lambda x : x[0]**2

x = gradientDescent(f,[2],0.4,0.4,0.000001,10000)
print(x, f(x))

Gradient descent took 12 iterations to converge
The norm of the gradient is 2.890914172880595e-07
[-0.19999984] 0.03999993769327916


3. [10 points] Implement a linear regression class in the style of scikit-learn regressors (i.e. as a Python class with fit and predict functions), with capabilities to use n random starting points and momentum (default to not using these features).

In [243]:
import copy
class question3:
    def __init__(self, use_momentum, num_starting_points, momentum_factor, learning_rate):
        self.num_starting_points = num_starting_points
        self.use_momentum = use_momentum
        self.momentum_factor = momentum_factor
        self.learning_rate = learning_rate
        
    def fit(self,X,y):
        self.X = np.array(X)
        self.y = np.array(y)
        n_classes = len(np.unique(y))

        # m = n_features
        n_samples,n_features = X.shape
        # set up random factors in weight w and bias b
        self.W = np.random.randn(n_features, n_classes, self.num_starting_points) # must be 3d due to multiple starting points
        self.b = np.zeros((n_classes, self.num_starting_points))

        # self.GradientDescent(self.X,self.y,self.w,self.b)

        if self.use_momentum:
            self.velocity_W = np.zeros((n_features, n_classes, self.num_starting_points))
            self.velocity_b = np.zeros((n_classes, self.num_starting_points))
        
        best_cost = float('inf') # intially save the largest number to save lower costs
        best_W = None
        best_b = None

        # Gradient descent for each starting points
        for start_point in range(self.num_starting_points):
            W = self.W[:, :, start_point]
            b = self.b[:, start_point]

            if self.use_momentum:
                velocity_W = self.velocity_W[:, :, start_point]
                velocity_b = self.velocity_b[:, start_point]

            for iteration in range(1000):
                dW, db = self.compute_gradient(X, y, W, b)

                if self.use_momentum:
                    # Update momentum terms
                    velocity_W = self.momentum_factor * velocity_W + (1 - self.momentum_factor) * dW
                    velocity_b = self.momentum_factor * velocity_b + (1 - self.momentum_factor) * db
                    
                    # Update weights and biases with momentum
                    W -= self.learning_rate * velocity_W
                    b -= self.learning_rate * velocity_b
                else:
                    # Update weights and biases without momentum
                    W -= self.learning_rate * dW
                    b -= self.learning_rate * db
                
                # Calculate cost to monitor the training process
                if iteration % 100 == 0:
                    cost = self.compute_cost(X, y, W, b)
                    print(f"Starting Point {start_point}, Iteration {iteration}, Cost: {cost}")

                # Keep track of the best weights and biases based on cost
                if cost < best_cost:
                    best_cost = cost
                    best_W = W
                    best_b = b
                    
            # Save the best weights and biases for current starting point
            self.W[:, :, start_point] = W
            self.b[:, start_point] = b

        # Set the best weights and bias
        self.W = best_W
        self.b = best_b
        
    
    def compute_cost(self,X,y,w,b):
        n = len(y)
        y_predict = np.dot(X, w) + b  # prediction
        cost = (1 / (2 * n)) * np.sum((y_predict - y) ** 2)
        return cost
    
    def GradientDescent(self,X,y,w,b):
        # initialize w and b
        self.velocity_w = np.zeros_like(self.w)
        self.velocity_b = 0

        for i in range(1000):
            dw, db = self.computeGradient(X, y, self.w, self.b)

            # set learning rate to 0.04 , momentum rate to 0.01
            self.w = self.w - dw * 0.04 - self.velocity_w * 0.01
            self.b = self.b - db * 0.04 - self.velocity_b * 0.01
            
            self.velocity_b = copy.deepcopy(self.b)
            self.velocity_w = copy.deepcopy(self.w)

            cost = self.computeCost(X, y, self.w, self.b)
            if i % 100 == 0:  # 
                print(f"Iteration {i}, Cost: {cost}")
            
            if np.abs(dw) < 0.1:
                print(self.w)
                print(f"Cost is: {cost}")
                print(i , " iterations")
                break
        return
    
    def compute_gradient(self,X,y,W,b):
        n_samples = X.shape[0]
        dw = 0
        db = 0
        z = np.dot(X, W) + b  # Shape: (n_samples, n_classes)
        
        # Compute gradients
        error = z - y
        dW = np.dot(X.T, error) / n_samples
        db = np.sum(error, axis=0) / n_samples

        return dw, db
    
    def predict(self, x):
        predict_y = np.dot(x,self.W) + self.b
        return predict_y

In [244]:
# Example input data
X = np.array([[1], [2], [3], [4], [5]])  # Input feature values
y = np.array([2, 4, 6, 8, 10])  # Target output values (simple linear relation: y = 2 * X)

# Initialize and fit the model
model = question3(num_starting_points = 5, momentum_factor = 0.09, use_momentum = True, learning_rate=0.02)
model.fit(X, y)

# Test the model's prediction
X_test = np.array([[6], [7], [8]])  # Test inputs
predictions = model.predict(X_test)
print("Predictions:", predictions)

Starting Point 0, Iteration 0, Cost: 108.86924305692764
Starting Point 0, Iteration 100, Cost: 4.125646765278932
Starting Point 0, Iteration 200, Cost: 2.2978774732733385
Starting Point 0, Iteration 300, Cost: 2.2659955937812186
Starting Point 0, Iteration 400, Cost: 2.2654394764176122
Starting Point 0, Iteration 500, Cost: 2.2654297760320956
Starting Point 0, Iteration 600, Cost: 2.2654296068277415
Starting Point 0, Iteration 700, Cost: 2.2654296038763007
Starting Point 0, Iteration 800, Cost: 2.2654296038248187
Starting Point 0, Iteration 900, Cost: 2.26542960382392
Starting Point 1, Iteration 0, Cost: 147.85881100595248
Starting Point 1, Iteration 100, Cost: 3.689014357780245
Starting Point 1, Iteration 200, Cost: 1.173260302377151
Starting Point 1, Iteration 300, Cost: 1.1293778671174108
Starting Point 1, Iteration 400, Cost: 1.1286124234047434
Starting Point 1, Iteration 500, Cost: 1.1285990717282972
Starting Point 1, Iteration 600, Cost: 1.1285988388342765
Starting Point 1, Itera

4. [10 points] Train and tune a linear regression model to predict death rates from cancer in US counties using the OLS Regression Challenge. During tuning, perform hyperparameter searches with n starting points, momentum, both, and neither. Use data from 30 random U.S. states for training, 10 random U.S. states for validation, and 10 random U.S. states for testing. Compare the results in terms of test accuracy, fit runtime, and predict runtime.

In [245]:
import pandas as pd
import numpy as np

# Read CSV file
data = pd.read_csv('cancer_reg.csv', encoding='latin-1') # Use latin-1 to avoid unicode errors

# Extract target and feature columns
y = data["TARGET_deathRate"]
X = data.drop(columns=["TARGET_deathRate"])

# Extract states from the 'Geography' column
states = data['Geography'].str.split(", ").str[1].unique() # Get unique state names

# Add the cleaned 'Geography' column to X
X['Geography'] = data['Geography'].str.split(", ").str[1]

# Set random seed for reproducibility
np.random.seed(42)

# Choose 30 | 10 | 10 states for training, validation, and testing
train_states = np.random.choice(states, 30, replace=False)
remaining_states = [state for state in states if state not in train_states]
val_states = np.random.choice(remaining_states, 10, replace=False)
test_states = [state for state in remaining_states if state not in val_states]

# Split data based on these states
train_data = data[data['Geography'].str.split(", ").str[1].isin(train_states)]
val_data = data[data['Geography'].str.split(", ").str[1].isin(val_states)]
test_data = data[data['Geography'].str.split(", ").str[1].isin(test_states)]

# Prepare the feature and target sets
X_train = train_data.drop(columns=["TARGET_deathRate", "Geography"])
y_train = train_data["TARGET_deathRate"]

X_val = val_data.drop(columns=["TARGET_deathRate", "Geography"])
y_val = val_data["TARGET_deathRate"]

X_test = test_data.drop(columns=["TARGET_deathRate", "Geography"])
y_test = test_data["TARGET_deathRate"]

# Display shapes to verify
# print(f"Training data shape: X_train={X_train.shape}, y_train={y_train.shape}")
# print(f"Validation data shape: X_val={X_val.shape}, y_val={y_val.shape}")
# print(f"Test data shape: X_test={X_test.shape}, y_test={y_test.shape}")

model2 = question3(num_starting_points = 5, momentum_factor = 0.09, use_momentum = True, learning_rate=0.02)

model2.fit(X_train,y_train)
model2.predict(X_val)

## couldn't solve due to lack of knowledge in pandas. I will work on it.

TypeError: can't multiply sequence by non-int of type 'float'

# Logistic Classification

5. [10 points] Derive an explicit formula for the gradient of the sum of squared errors loss for a logistic classifier with $k$
 classes for input data of dimension $d+1$ and one-hot labels. [Hint. Use the softmax function instead of sigmoid.]

In [106]:
class softmax_regression:
    def __init__(self, learning_rate,max_iter):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
    def fit(self,X,y):
        # m = number of samples
        # n = number of features
        m,n = X.shape 
        # get number of unique inputs
        n_classes = len(np.unique(y))

        # initialize weight + bias
        self.W = np.random.randn(n, n_classes)
        self.b = np.zeros((1, n_classes))

        # convert y to one-hot labels
        y_one_hot = np.eye(n_classes)[y]

         # Gradient Descent Loop
        for i in range(self.max_iter):
            dW, db = self.compute_gradient(X, y_one_hot, self.W, self.b)
            
            # Update weights and bias
            self.W -= self.learning_rate * dW
            self.b -= self.learning_rate * db
            
            # Optionally print the cost every 100 iterations
            if i % 100 == 0:
                cost = self.compute_cost(X, y_one_hot, self.W, self.b)
                print(f"Iteration {i}, Cost: {cost}")

    def compute_gradient(self, X, y_one_hot, W, b):
        n_samples = X.shape[0]
        z = np.dot(X, W) + b 

        # make prediction with softmax function
        # y_pred.shape = m,n
        y_pred = self.softmax(z)

        error = y_pred - y_one_hot
        dW = np.dot(X.T, error) / n_samples
        db = np.sum(error, axis=0, keepdims=True) / n_samples

        return dW, db
        
    def softmax(self, z):

        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability

        return exp_z / np.sum(exp_z, axis=1, keepdims=True)  # Compute probabilities

    def compute_cost(self, X, y_one_hot, W, b):
        n_samples = X.shape[0]
        z = np.dot(X, W) + b  # (shape: [n_samples, n_classes])
        y_pred = self.softmax(z)  # Apply softmax to get class probabilities
            
        # Sum of Squared Errors loss / num of samples
        cost = (1 / 2) * np.sum((y_one_hot - y_pred) ** 2) / n_samples
        return cost
    def predict(self, X):

        z = np.dot(X, self.W) + self.b  # Linear combination
        y_pred = self.softmax(z)  # Apply softmax to get probabilities
        return np.argmax(y_pred, axis=1)  # Return the index of the highest probability



In [107]:
np.random.seed(42)
X = np.random.rand(100, 3)  # 100 samples, 3 features (dimension d + 1)
y = np.random.randint(0, 3, 100)  # 3 classes (k = 3)

# Create and train the model
model = softmax_regression(learning_rate=0.1, max_iter=1000)
model.fit(X, y)

# Predict on training data
predictions = model.predict(X)
print("Predicted class labels:", predictions)

Iteration 0, Cost: 0.36318778042735717
Iteration 100, Cost: 0.34358702306708594
Iteration 200, Cost: 0.334286064837539
Iteration 300, Cost: 0.3285529132497018
Iteration 400, Cost: 0.32505085543734447
Iteration 500, Cost: 0.32289269592887165
Iteration 600, Cost: 0.3215381261306709
Iteration 700, Cost: 0.32066884563250925
Iteration 800, Cost: 0.3200982424761425
Iteration 900, Cost: 0.3197157935470677
Predicted class labels: [0 1 0 1 1 0 0 0 0 0 1 2 1 1 0 1 1 2 2 1 0 0 0 2 0 2 1 1 0 2 0 0 1 0 0 0 1
 0 2 0 2 0 2 1 0 0 0 0 1 1 1 2 2 1 1 1 0 0 1 0 0 1 1 0 2 0 2 0 1 1 0 0 0 2
 0 0 2 0 0 0 2 1 2 1 1 2 1 1 2 1 2 1 2 2 1 0 0 0 0 0]


6. [5 points] Repeat Problem 5 with the cross-entropy loss.

In [108]:
# for question 6, the code is similar to question 5 and I have just changed the compute_cost function.

class cross_entry_regression:
    def __init__(self, learning_rate,max_iter):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
    def fit(self,X,y):
        # m = number of samples
        # n = number of features
        m,n = X.shape 
        # get number of unique inputs
        n_classes = len(np.unique(y))

        # initialize weight + bias
        self.W = np.random.randn(n, n_classes)
        self.b = np.zeros((1, n_classes))

        # convert y to one-hot labels
        y_one_hot = np.eye(n_classes)[y]

         # Gradient Descent Loop
        for i in range(self.max_iter):
            dW, db = self.compute_gradient(X, y_one_hot, self.W, self.b)
            
            # Update weights and bias
            self.W -= self.learning_rate * dW
            self.b -= self.learning_rate * db
            
            # Optionally print the cost every 100 iterations
            if i % 100 == 0:
                cost = self.compute_cost_cross_entry(X, y_one_hot, self.W, self.b).T
                print(f"Iteration {i}, Cost: {cost}")

    def compute_gradient(self, X, y_one_hot, W, b):
        n_samples = X.shape[0]
        z = np.dot(X, W) + b 

        # make prediction with softmax function
        # y_pred.shape = m,n
        y_pred = self.softmax(z)

        error = y_pred - y_one_hot
        dW = np.dot(X.T, error) / n_samples
        db = np.sum(error, axis=0, keepdims=True) / n_samples

        return dW, db
        
    def softmax(self, z):

        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability

        return exp_z / np.sum(exp_z, axis=1, keepdims=True)  # Compute probabilities

    def compute_cost_cross_entry(self, X, y_one_hot, W, b):
        n_samples = X.shape[0]
        z = np.dot(X, W) + b  # (shape: [n_samples, n_classes])
        y_pred = self.softmax(z)  # Apply softmax to get class probabilities
            
        # Sum of Squared Errors loss / num of samples
        cost = np.sum(np.dot(y_one_hot.T,y_pred)) / n_samples
        return cost
    
    def predict(self, X):

        z = np.dot(X, self.W) + self.b  # Linear combination
        y_pred = self.softmax(z)  # Apply softmax to get probabilities
        return np.argmax(y_pred, axis=1)  # Return the index of the highest probability

In [109]:
np.random.seed(42)
X = np.random.rand(100, 3)  # 100 samples, 3 features (dimension d + 1)
y = np.random.randint(0, 3, 100)  # 3 classes (k = 3)

# Create and train the model
model = cross_entry_regression(learning_rate=0.1, max_iter=1000)
model.fit(X, y)

# Predict on training data
predictions = model.predict(X)
print("Predicted class labels:", predictions)

Iteration 0, Cost: 1.0
Iteration 100, Cost: 1.0
Iteration 200, Cost: 0.9999999999999999
Iteration 300, Cost: 1.0
Iteration 400, Cost: 0.9999999999999999
Iteration 500, Cost: 1.0
Iteration 600, Cost: 1.0
Iteration 700, Cost: 1.0
Iteration 800, Cost: 1.0
Iteration 900, Cost: 1.0
Predicted class labels: [0 1 0 1 1 0 0 0 0 0 1 2 1 1 0 1 1 2 2 1 0 0 0 2 0 2 1 1 0 2 0 0 1 0 0 0 1
 0 2 0 2 0 2 1 0 0 0 0 1 1 1 2 2 1 1 1 0 0 1 0 0 1 1 0 2 0 2 0 1 1 0 0 0 2
 0 0 2 0 0 0 2 1 2 1 1 2 1 1 2 1 2 1 2 2 1 0 0 0 0 0]


7. [10 points] Write a multi-class logistic classifier (for $k$ classes) in the style of scikit-learn classifiers (i.e. as a Python class with fit and predict functions), optimized by gradient descent with options to use multiple starting points and momentum.

In [170]:

class MultiClassLogisticClassifier:
    def __init__(self, learning_rate = 0.01, max_iter = 1000, num_starting_points = 1, use_momentum = False, momentum_factor = 0.9):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.num_starting_points = num_starting_points
        self.use_momentum = use_momentum
        self.momentum_factor = momentum_factor

    def fit(self, X, y):
        
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y)) # get unique input of y's
        
        # Initialize weights and biases for multiple starting points
        self.W = np.random.randn(n_features, n_classes, self.num_starting_points) # must be 3d due to multiple starting points
        self.b = np.zeros((n_classes, self.num_starting_points))

        # Initialize momentum for gradient descent
        if self.use_momentum:
            self.velocity_W = np.zeros((n_features, n_classes, self.num_starting_points))
            self.velocity_b = np.zeros((n_classes, self.num_starting_points))
        
        # Convert y to one-hot encoding
        y_one_hot = np.eye(n_classes)[y]
        
        # save the best weight and bias
        best_cost = float('inf') # intially save the largest number to save lower costs
        best_W = None
        best_b = None

        # Gradient Descent for each starting points
        for start_point in range(self.num_starting_points):
            W = self.W[:, :, start_point]
            b = self.b[:, start_point]

            if self.use_momentum:
                velocity_W = self.velocity_W[:, :, start_point]
                velocity_b = self.velocity_b[:, start_point]

            for iteration in range(self.max_iter):
                # Compute gradient
                dW, db = self.compute_gradient(X, y_one_hot, W, b)
                
                if self.use_momentum:
                    # Update momentum terms
                    velocity_W = self.momentum_factor * velocity_W + (1 - self.momentum_factor) * dW
                    velocity_b = self.momentum_factor * velocity_b + (1 - self.momentum_factor) * db
                    
                    # Update weights and biases with momentum
                    W -= self.learning_rate * velocity_W
                    b -= self.learning_rate * velocity_b
                else:
                    # Update weights and biases without momentum
                    W -= self.learning_rate * dW
                    b -= self.learning_rate * db

                # Calculate cost to monitor the training process
                if iteration % 100 == 0:
                    cost = self.compute_cost(X, y_one_hot, W, b)
                    print(f"Starting Point {start_point}, Iteration {iteration}, Cost: {cost}")

                # Keep track of the best weights and biases based on cost
                if cost < best_cost:
                    best_cost = cost
                    best_W = W
                    best_b = b

            # Save the best weights and biases for current starting point
            self.W[:, :, start_point] = W
            self.b[:, start_point] = b
        
        # Set the best weights and bias
        self.W = best_W
        self.b = best_b

    def compute_gradient(self, X, y_one_hot, W, b):
       
        n_samples = X.shape[0] # get  num of inputs
        
        # Compute predictions
        z = np.dot(X, W) + b  # Shape: (n_samples, n_classes)
        y_pred = self.softmax(z)
        
        # Compute gradients
        error = y_pred - y_one_hot
        dW = np.dot(X.T, error) / n_samples
        db = np.sum(error, axis=0) / n_samples
        
        return dW, db

    def softmax(self, z):

        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def compute_cost(self, X, y_one_hot, W, b):
        
        n_samples = X.shape[0]
        
        # Compute predictions
        z = np.dot(X, W) + b
        y_pred = self.softmax(z)
        
        # Compute cross-entropy loss
        loss = -np.sum(y_one_hot * np.log(y_pred + 1e-15)) / n_samples
        return loss

    def predict(self, X):
        
        z = np.dot(X, self.W) + self.b
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)

In [171]:
# Example dataset
np.random.seed(42)
X = np.random.rand(100, 3)  # 100 samples with 3 features
y = np.random.randint(0, 3, 100)  # 3 classes (0, 1, 2)

# Create and train the model with multiple starting points and momentum
model = MultiClassLogisticClassifier(learning_rate=0.1, max_iter=1000, num_starting_points=3, use_momentum=True)
model.fit(X, y)

# Make predictions
predictions = model.predict(X)
print("Predicted class labels:", predictions)

Starting Point 0, Iteration 0, Cost: 1.5516518996491795
Starting Point 0, Iteration 100, Cost: 1.1357508614058431
Starting Point 0, Iteration 200, Cost: 1.1009967161381646
Starting Point 0, Iteration 300, Cost: 1.0804510228285107
Starting Point 0, Iteration 400, Cost: 1.0684170782111986
Starting Point 0, Iteration 500, Cost: 1.0613512391644835
Starting Point 0, Iteration 600, Cost: 1.0571586300721088
Starting Point 0, Iteration 700, Cost: 1.0546304898249967
Starting Point 0, Iteration 800, Cost: 1.0530751551267858
Starting Point 0, Iteration 900, Cost: 1.0520963522178368
Starting Point 1, Iteration 0, Cost: 1.494476191392832
Starting Point 1, Iteration 100, Cost: 1.1159350287535643
Starting Point 1, Iteration 200, Cost: 1.0893932161585522
Starting Point 1, Iteration 300, Cost: 1.0736510803845964
Starting Point 1, Iteration 400, Cost: 1.0643240832837555
Starting Point 1, Iteration 500, Cost: 1.058778943052082
Starting Point 1, Iteration 600, Cost: 1.0554612090332263
Starting Point 1, It

8. [10 points] Train and tune a logistic classifier model to predict the digits of the MNIST dataset. During tuning, perform hyperparameter searches with $n$ starting points, momentum, both, and neither. Use a 60% / 20% / 20% for training / validation / testing split. Compare the results in terms of test performance (confusion matrix and classification report), fit runtime, and predict runtime.

In [176]:
class FeedforwardNeuralNetworkSGD:
    def __init__(self, learning_rate = 0.01, max_iter = 1000, num_starting_points = 1, use_momentum = False, momentum_factor = 0.9, batchSize = 32):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.num_starting_points = num_starting_points
        self.use_momentum = use_momentum
        self.momentum_factor = momentum_factor
        self.batchSize = batchSize

    def fit(self, X, y):
        
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y)) # get unique input of y's
        
        # Initialize weights and biases for multiple starting points
        self.W = np.random.randn(n_features, n_classes, self.num_starting_points) # must be 3d due to multiple starting points
        self.b = np.zeros((n_classes, self.num_starting_points))

        # Initialize momentum for gradient descent
        if self.use_momentum:
            self.velocity_W = np.zeros((n_features, n_classes, self.num_starting_points))
            self.velocity_b = np.zeros((n_classes, self.num_starting_points))
        
        # Convert y to one-hot encoding
        y_one_hot = np.eye(n_classes)[y]
        
        # save the best weight and bias
        best_cost = float('inf') # intially save the largest number to save lower costs
        best_W = None
        best_b = None

        # Gradient Descent for each starting points
        for start_point in range(self.num_starting_points):
            W = self.W[:, :, start_point]
            b = self.b[:, start_point]

            if self.use_momentum:
                velocity_W = self.velocity_W[:, :, start_point]
                velocity_b = self.velocity_b[:, start_point]

            for iteration in range(self.max_iter):
                # Compute gradient
                dW, db = self.compute_gradient(X, y_one_hot, W, b)
                
                if self.use_momentum:
                    # Update momentum terms
                    velocity_W = self.momentum_factor * velocity_W + (1 - self.momentum_factor) * dW
                    velocity_b = self.momentum_factor * velocity_b + (1 - self.momentum_factor) * db
                    
                    # Update weights and biases with momentum
                    W -= self.learning_rate * velocity_W
                    b -= self.learning_rate * velocity_b
                else:
                    # Update weights and biases without momentum
                    W -= self.learning_rate * dW
                    b -= self.learning_rate * db

                # Calculate cost to monitor the training process
                if iteration % 100 == 0:
                    cost = self.compute_cost(X, y_one_hot, W, b)
                    print(f"Starting Point {start_point}, Iteration {iteration}, Cost: {cost}")

                # Keep track of the best weights and biases based on cost
                if cost < best_cost:
                    best_cost = cost
                    best_W = W
                    best_b = b

            # Save the best weights and biases for current starting point
            self.W[:, :, start_point] = W
            self.b[:, start_point] = b
        
        # Set the best weights and bias
        self.W = best_W
        self.b = best_b

    def compute_gradient(self, X, y_one_hot, W, b):
       
        n_samples = X.shape[0] # get  num of inputs
        
        # Compute predictions
        z = np.dot(X, W) + b  # Shape: (n_samples, n_classes)
        y_pred = self.softmax(z)
        
        # Compute gradients
        error = y_pred - y_one_hot
        dW = np.dot(X.T, error) / n_samples
        db = np.sum(error, axis=0) / n_samples
        
        return dW, db

    def softmax(self, z):

        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def compute_cost(self, X, y_one_hot, W, b):
        
        n_samples = X.shape[0]
        
        # Compute predictions
        z = np.dot(X, W) + b
        y_pred = self.softmax(z)
        
        # Compute cross-entropy loss
        loss = -np.sum(y_one_hot * np.log(y_pred + 1e-15)) / n_samples
        return loss

    def predict(self, X):
        
        z = np.dot(X, self.W) + self.b
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)



#############

from keras.datasets import mnist
from keras.utils import to_categorical

# load the full MNIST dataset: both data and labels
((trainX, trainY), (testX, testY)) = mnist.load_data()

# scale the data to values in [0,1]
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

# reshape the data
trainX = trainX.reshape([60000, 28*28])
testX = testX.reshape([10000, 28*28])

# convert the digits to one-hot vectors
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

mnist_model = FeedforwardNeuralNetworkSGD([784, 32, 16, 10], 0.5, 32)

mnist_model.fit(trainX)
mnist_model.fit(trainX, trainY, 100, 1)



# couldn't solve due to lack of keras knowledge

TypeError: FeedforwardNeuralNetworkSGD.fit() missing 1 required positional argument: 'y'